In [1]:
def xml(url):
    import pandas as pd
    import numpy as np
    import elementpath
    from xml.etree import ElementTree as ET
    mytree = ET.parse(url)
    myroot = mytree.getroot()
    
    Job = []
    for job_bd in myroot.iter('COMBINED-PAYMENT-HISTORY'):
        Job.append(job_bd.text)
    newlist = []
    for word in Job:
        word = word.split("|")
        newlist.append(word)
    df = pd.DataFrame(newlist)
    
    for i in df.columns.values:
        df[i] = df[i].str[9:12]
    df = df.replace(regex=[r'\D+'], value='')
    for i in df.columns.values:
        df[i] = pd.to_numeric(df[i])
    df.replace(to_replace=0.0, value=np.nan,inplace=True)
    df['new'] = df.apply(lambda _: '', axis=1)
    
    Og = df.melt(id_vars=["new"]).value.dropna()
    Og = pd.DataFrame(Og)
    Og.dropna(inplace=True)
    Og.reset_index(inplace=True, drop=True)
    
    amt = []
    for i in myroot.iter('DISBURSED-AMT'):
        amt.append(i.text.replace(",",''))
    amount=[int(x) for x in amt]
    Total_amount=[sum(amount)]
    
    Total_amount=pd.DataFrame(Total_amount)
    Total_amount.columns=['Total DISBURSED-AMT']
    orginal_data = pd.concat([Og,Total_amount],axis="columns")
    return(orginal_data)

In [2]:
Customer_1 = xml('C:\\Users\\soura\\OneDrive\\Desktop\\Coding_data\\customer14235_loan14235_crif_report.html.xml')
Customer_2 = xml('C:\\Users\\soura\\OneDrive\\Desktop\\Coding_data\\customer16475_loan16475_crif_report.html.xml')
Customer_3 = xml('C:\\Users\\soura\\OneDrive\\Desktop\\Coding_data\\customer40409_loan40409_crif_report.html.xml')
Customer_4 = xml('C:\\Users\\soura\\OneDrive\\Desktop\\Coding_data\\customer773504_loan774538_crif_report.html.xml')
Customer_5 = xml('C:\\Users\\soura\\OneDrive\\Desktop\\Coding_data\\customer787561_loan788638_crif_report.html.xml')
Customer_6 = xml('C:\\Users\\soura\\OneDrive\\Desktop\\Coding_data\\customer794397_loan795497_crif_report.html.xml')
Customer_7 = xml('C:\\Users\\soura\\OneDrive\\Desktop\\Coding_data\\customer898231_loan899591_crif_report.html.xml')
Customer_8 = xml('C:\\Users\\soura\\OneDrive\\Desktop\\Coding_data\\customer1113697_loan1115483_crif_report.html.xml')
Customer_9 = xml('C:\\Users\\soura\\OneDrive\\Desktop\\Coding_data\\customer1129550_loan1131339_crif_report.html.xml')
Customer_10 = xml('C:\\Users\\soura\\OneDrive\\Desktop\\Coding_data\\customer1195586_loan1197471_crif_report.html.xml')

In [3]:
import pandas as pd
DPD= pd.concat([Customer_1,Customer_2,Customer_3,Customer_4,Customer_5,Customer_6,Customer_7,Customer_8,Customer_9,Customer_10],axis="columns")

In [4]:
DPD.to_excel('DPD.xlsx')

In [5]:
df=pd.read_excel('C:\\Users\\soura\\DPD.xlsx')

In [6]:
df.drop(columns='Unnamed: 0',inplace =True)

In [7]:
df.head()

,value_1,Total DISBURSED-AMT_1,value_2,Total DISBURSED-AMT_2,value_3,Total DISBURSED-AMT_3,value_4,Total DISBURSED-AMT_4,value_5,Total DISBURSED-AMT_5,value_6,Total DISBURSED-AMT_6,value_7,Total DISBURSED-AMT_7,value_8,Total DISBURSED-AMT_8,value_9,Total DISBURSED-AMT_9,value_10,Total DISBURSED-AMT_10
0,60.0,9485895.0,77.0,2868467.0,93.0,1075308.0,141.0,795601.0,89.0,810839.0,171.0,910791.0,88,13005665.0,175.0,1110420.0,141.0,795601.0,90.0,8118705.0
1,180.0,NaN,52.0,NaN,5.0,NaN,53.0,NaN,89.0,NaN,52.0,NaN,67,NaN,114.0,NaN,53.0,NaN,90.0,NaN
2,120.0,NaN,78.0,NaN,63.0,NaN,93.0,NaN,31.0,NaN,60.0,NaN,43,NaN,172.0,NaN,93.0,NaN,80.0,NaN
3,30.0,NaN,49.0,NaN,329.0,NaN,334.0,NaN,77.0,NaN,179.0,NaN,67,NaN,210.0,NaN,334.0,NaN,66.0,NaN
4,120.0,NaN,46.0,NaN,28.0,NaN,125.0,NaN,31.0,NaN,60.0,NaN,67,NaN,73.0,NaN,125.0,NaN,92.0,NaN


In [8]:
#percentage of trades are with 30+ DPD
percentage = []
for i in [item for item in df.columns.values if item.startswith('v')]:
    percentage.append((len(df.loc[df[i]>30])/len(df)*100,i))
print(max(percentage))

(65.78947368421053, 'value_7')


In [9]:
#sum of total disbursed amount for all loans for each customer
Sum_amount = [col for col in df if col.startswith('T')]
print(df[Sum_amount].iloc[0])

Total DISBURSED-AMT_1      9485895.0
Total DISBURSED-AMT_2      2868467.0
Total DISBURSED-AMT_3      1075308.0
Total DISBURSED-AMT_4       795601.0
Total DISBURSED-AMT_5       810839.0
Total DISBURSED-AMT_6       910791.0
Total DISBURSED-AMT_7     13005665.0
Total DISBURSED-AMT_8      1110420.0
Total DISBURSED-AMT_9       795601.0
Total DISBURSED-AMT_10     8118705.0
Name: 0, dtype: float64


In [10]:
#number of months of 30+ due per trade
p=[]
for i in [item for item in df.columns.values if item.startswith('v')]:
    p.append((len(df.loc[df[i]>30]),i))
p

[(17, 'value_1'),
 (12, 'value_2'),
 (13, 'value_3'),
 (21, 'value_4'),
 (7, 'value_5'),
 (15, 'value_6'),
 (25, 'value_7'),
 (16, 'value_8'),
 (21, 'value_9'),
 (16, 'value_10')]

In [11]:
#maximum number of months of 30+ due per trade
print(max(p))

(25, 'value_7')
